# Adv. Data Mining
### Ryan Durfey
## Problem Set 7: Part 3 - Python

## Chapter 9
## Exercise 8

In [1]:
%matplotlib inline

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# this is a new import
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn import datasets
import seaborn as sns

from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

In [5]:
OJ = pd.read_csv("./OJ.csv")
OJ.head()

,Unnamed: 0,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,1,CH,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,No,0.000000,0.000000,0.24,1
1,2,CH,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,No,0.150754,0.000000,0.24,1
2,3,CH,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,No,0.000000,0.091398,0.23,1
3,4,MM,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,No,0.000000,0.000000,0.00,1
4,5,CH,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,Yes,0.000000,0.000000,0.00,0


#8a

In [6]:
## make into pd.Dataframe so that we can drop/select columns
OJ_DF = pd.DataFrame(OJ,columns=['Purchase','WeekofPurchase','StoreID','PriceCH','PriceMM','DiscCH','DiscMM','SpecialCH','SpecialMM','LoyalCH','SalePriceMM','SalePriceCH','PriceDiff','Store7','PctDiscMM','PctDiscCH','ListPriceDiff','STORE'])

## separate X and y
X = OJ_DF.drop(['Purchase','Store7'],axis=1) # python hates the string variables, so must drop Store7
y = OJ_DF.Purchase

## create training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = (len(OJ)-800))


In [7]:
def train_svm(kernel="linear", C=1.0, gamma=0.0):

    svm = SVC(kernel=kernel, C=C, gamma=gamma)
    svm.fit(X_train, y_train)
    
    # predict on training set
    y_preds_train = svm.predict(X_train)

    # predict on the test set
    y_preds_test = svm.predict(X_test)
    # get the distances from the hyperplane, the sign of which
    # is the prediction above
    y_dists = svm.decision_function(X_test)
    
    return (svm, y_preds_train, y_preds_test, y_dists)

#8b

In [8]:
svm_linear, y_preds_train, y_preds_test, y_dists_linear = train_svm(kernel="linear",C=0.01)
svm_linear.n_support_

## It looks like the Support Vector Classifier creates 609 support vectors, with 306 and 303 belonging to each class

array([309, 308])

In [9]:
svm_linear.n_support_

array([309, 308])

#8c

In [10]:
## training error rate
train_error = float(sum(y_preds_train!=y_train))/float(len(X_train))
print "Train Error is %s" % train_error

## test error rate
test_error = float(sum(y_preds_test!=y_test))/float(len(X_test))
print "Test Error is %s" % test_error

Train Error is 0.23875
Test Error is 0.274074074074


#8d

In [12]:
#C_param = 10. ** np.arange(-2, 9)
#train_err = []
#test_err = []
#for i in C_param:
#    svm = SVC(kernel="linear", C=i)
#    svm.fit(X_train,y_train)
#    train_err.append(float(sum(svm.predict(X_train) != y_train))/float(len(X_train)))
#    test_err.append(float(sum(svm.predict(X_test) != y_test))/float(len(X_test)))


C_range = 10. ** np.arange(-2, 9)
gamma_range = 10. ** np.arange(-5, 4)

param_grid = dict(gamma=gamma_range, C=C_range)

grid = GridSearchCV(SVC(), param_grid=param_grid, cv=StratifiedKFold(y=y))

grid.fit(X, y)

print("The best classifier is: ", grid.best_estimator_)

('The best classifier is: ', SVC(C=100000.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=1.0000000000000001e-05, kernel='rbf', max_iter=-1,
  probability=False, random_state=None, shrinking=True, tol=0.001,
  verbose=False))


In [ ]:
plt.plot(C_param,train_err)
plt.title('Training Error as a function of C')
plt.xlabel('C')
plt.ylabel('Training Error')

In [ ]:
plt.plot(C_param,test_err)
plt.title('Test Error as a function of C')
plt.xlabel('C')
plt.ylabel('Test Error')

In [ ]:
?SVC
